# Imports

Import python libraries as well as the self written FERMI library.

In [ ]:
import sys, os
from os.path import join, split
from getpass import getuser
from glob import glob
from time import strftime
from tqdm.auto import tqdm
from importlib import reload
from copy import deepcopy

# Data
import numpy as np
import xarray as xr
import pandas as pd
import h5py

# Images
import imageio
from imageio import imread

# Plotting
import matplotlib.pyplot as plt
from matplotlib.image import NonUniformImage
import matplotlib.gridspec as gridspec
from matplotlib.path import Path

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
import process_FERMI as pf
from process_FERMI import interactive
from proces_FERMI.interactive import cimshow

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# interactive plotting
import ipywidgets
%matplotlib widget
plt.rcParams["figure.constrained_layout.use"] = True

# Auto formatting of cells
%load_ext jupyter_black

## Functions

In [ ]:
def create_folder(folder):
    """
    Creates input folder if it does not exist yet
    """

    if not (os.path.exists(folder)):
        print("Creating folder " + folder)
        os.makedirs(folder)
    return folder

In [ ]:
def get_hysteresis(exp, dark, mask, magnet_string="magnet_mean"):
    print("Loading images and summing the scattering.")
    total_intensities = np.empty(len(exp))
    for i in tqdm(range(len(exp))):
        imi, _ = pf.loadh5(
            exp.iloc[i]["filename"], extra_keys=["alignz", "PAM/FQPDSum"]
        )
        total_intensities[i] = np.sum(
            (imi.astype(float) - dark.astype(float)) * mask.astype(float)
        )

    print("Averaging the images at the same magnetic field.")
    total_intensity = []
    magnet = np.unique(exp[magnet_string])
    for m in tqdm(magnet):
        idx = np.where(exp[magnet_string] == m)[0]
        total_intensity.append(np.mean(total_intensities[idx]))

    return np.array(total_intensity), magnet

In [ ]:
def get_metadata(fname, extra_keys, magnet_function, dataname=None, darkname=None):
    if dataname is not None:
        fname = fnm

    exp = pf.get_exp_dataframe(fname, keys=extra_keys)
    for k in ["xgm_UH", "xgm_SH", "diode_sum"]:
        exp[k + "_sum"] = exp[k].apply(np.sum)

    exp["diode_sum_mean"] = exp.diode_sum.apply(np.mean)
    exp["diode_sum_std"] = exp.diode_sum.apply(np.std)
    exp["IR_mean"] = exp.IR.apply(np.mean)
    exp["IR_std"] = exp.IR.apply(np.std)
    exp["magnet_mean"] = exp.magnet.apply(magnet_function)
    exp["magnet_mean"] = exp.magnet_mean.apply(np.round, args=(3,))
    exp["bunchid"] = exp.bunches.apply(lambda l: l[-1])

    if darkname is not None:
        exp_bg = pf.get_exp_dataframe(darkname, keys=extra_keys)
        exp_bg["bunchid"] = exp.bunches.apply(lambda l: l[-1])
        exp_bg = exp_bg.sort_values("time")

        darkfolder = datafolder + "_BG"
        darks = []
        dflist = glob(darkfolder + "/**/*h5", recursive=True)
        print(f"found {len(dflist)} dark files")
        for f in dflist:
            _dark = pf.loadh5(f)[0]
            darks.append(_dark)
        dark = np.mean(darks, axis=0).astype(float)

        return (exp, dark)
    return dark

In [ ]:
def preprocess_exp_OF(datafolder, keys=None, sort=False):
    # Loading experiment data
    extension = "_OF"
    exp = pf.get_exp_dataframe(datafolder + extension, keys=keys)
    for k in ["xgm_UH", "xgm_SH", "diode_sum"]:
        exp[k + "_sum"] = exp[k].apply(np.sum)

    exp["diode_sum_mean"] = exp.diode_sum.apply(np.mean)
    exp["diode_sum_std"] = exp.diode_sum.apply(np.std)
    exp["IR_mean"] = exp.IR.apply(np.mean)
    exp["IR_std"] = exp.IR.apply(np.std)
    exp["magnet_mean"] = exp.magnet.apply(np.mean)
    exp["magnet_mean"] = exp.magnet_mean.apply(np.round, args=(3,))
    exp["bunchid"] = exp.bunches.apply(lambda l: l[-1])

    if sort is True:
        exp = exp.sort_values(scan_axis)

    # Loading image data
    exp["images"] = [
        pf.loadh5(fname, extra_keys=["alignz", "PAM/FQPDSum"])[0]
        for fname in exp["filename"]
    ]

    return exp


def preprocess_exp_BG(datafolder, keys=None):
    # Loading background exp dataframe
    extension = "_BG"
    exp_bg = pf.get_exp_dataframe(datafolder + extension, keys=keys)
    exp_bg["bunchid"] = exp.bunches.apply(lambda l: l[-1])
    exp_bg = exp_bg.sort_values("time")

    # Loading background data
    exp_bg["images"] = [
        pf.loadh5(fname, extra_keys=["alignz", "PAM/FQPDSum"])[0]
        for fname in exp_bg["filename"]
    ]

    return exp_bg

In [ ]:
def norm(x):
    """
    Normalize data to [-1,1] range for hysteresis
    """
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [ ]:
# Draw circle mask
def circle_mask(shape, center, radius, sigma="none"):
    """
    Draws circle mask with option to apply gaussian filter for smoothing

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    center : int tuple
        center coordinates (ycenter,xcenter)
    radius : scalar
        radius of mask in px. Care: diameter is always (2*radius+1) px
    sigma : scalar
        std of gaussian filter

    Output
    ======
    mask: array
        binary mask, or smoothed binary mask
    ======
    author: ck 2022
    """

    # setup array
    x = np.linspace(0, shape[1] - 1, shape[1])
    y = np.linspace(0, shape[0] - 1, shape[0])
    X, Y = np.meshgrid(x, y)

    # define circle
    mask = np.sqrt(((X - center[1]) ** 2 + (Y - center[0]) ** 2)) <= (radius)
    mask = mask.astype(float)

    # smooth aperture
    if sigma != "none":
        mask = scipy.ndimage.filters.gaussian_filter(mask, sigma)

    return mask


def create_single_polygon_mask(shape, coordinates):
    """
    Creates a polygon mask from coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points [[yc_1,xc_1],[yc_2,xc_2],...]


    Output
    ======
    mask: array
        binary mask where filled polygon is "1"
    ======
    author: ck 2023
    """

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x, y = x.flatten(), y.flatten()

    points = np.vstack((x, y)).T

    path = Path(coordinates)
    mask = path.contains_points(points)
    mask = mask.reshape(shape)
    return mask


def create_polygon_mask(shape, coordinates):
    """
    Creates multiple polygon masks from set of coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points for multiple polygons
        [[[yc_1,xc_1],[yc_2,xc_2],...],[[yc_1,xc_1],[yc_2,xc_2],...]]

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    if len(coordinates) == 1:
        mask = create_single_polygon_mask(shape, coordinates[0])
    elif len(coordinates) > 1:
        mask = np.zeros(shape)
        for coord in coordinates:
            mask = mask + create_single_polygon_mask(shape, coord)
            mask[mask > 1] = 1

    return mask


def load_poly_masks(polygon_name_list, shape):
    """
    Loads set of polygon masks based on stored coordinates

    Parameter
    =========
    polygon_name_list : list
        shape/dimension of output array


    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    mask = []
    for polygon_name in polygon_name_list:
        coord = load_poly_coordinates(polygon_name)
        mask.append(create_polygon_mask(shape, [coord]).astype(float))

    mask = np.array(mask)
    mask = np.sum(mask, axis=0)
    mask[mask > 1] = 1

    return mask

# Experimental details

In [ ]:
# Define basic folders
BASEFOLDER = r"/net/online4diproi/store/"
PROPOSAL = "20224053"
basefolder = join(BASEFOLDER, PROPOSAL)
USER = getuser()

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.103,  # ccd to sample distance
    "px_size": 11e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)

# General saving folder
folder_general = create_folder(join(basefolder, "results", "processed"))
print("Output Folder: %s" % folder_general)

# Load Data

### Scan

In [ ]:
# Define for loading
sample = "S2205ref"
membrane = "I0"
scan_id = 11
scan = f"%s_HystScan_Inverted_%03d" % (membrane, scan_id)
scan_axis = "magnet_mean"
extension = "_OF"

# Folder for loading
samplefolder = join(sample, scan)
datafolder = join(basefolder, samplefolder)
extra_keys = {
    "diode_sum": "PAM/FQPDSum",
    "IR": "Laser/Energy1",
    "magnet": "DPI/CoilCurrent",
    "bunches": "bunches",
    "time": "",
}

# Create savefolder
fsave = create_folder(join(folder_general, sample, membrane))

# Loading experiment data
exp = pf.get_exp_dataframe(datafolder + extension, keys=extra_keys)
for k in ["xgm_UH", "xgm_SH", "diode_sum"]:
    exp[k + "_sum"] = exp[k].apply(np.sum)

exp["diode_sum_mean"] = exp.diode_sum.apply(np.mean)
exp["diode_sum_std"] = exp.diode_sum.apply(np.std)
exp["IR_mean"] = exp.IR.apply(np.mean)
exp["IR_std"] = exp.IR.apply(np.std)
exp["magnet_mean"] = exp.magnet.apply(np.mean)
exp["magnet_mean"] = exp.magnet_mean.apply(np.round, args=(3,))
exp["bunchid"] = exp.bunches.apply(lambda l: l[-1])
# exp = exp.sort_values(scan_axis)

# Add wavelength
experimental_setup["lambda"] = exp["wavelength"][0] * 1e-9

# Loading image data
exp["images"] = [
    pf.loadh5(fname, extra_keys=["alignz", "PAM/FQPDSum"])[0]
    for fname in exp["filename"]
]
print("Data loaded!")

In [ ]:
# What did you scan?
fig, ax = plt.subplots()
ax.plot(np.arange(len(exp)), exp[scan_axis], "-o")
ax.set_xlabel("Index")
ax.set_ylabel(scan_axis)

### Dark images

In [ ]:
# dark image
extension = "_BG"
exp_bg = pf.get_exp_dataframe(datafolder + extension, keys=extra_keys)
exp_bg["bunchid"] = exp.bunches.apply(lambda l: l[-1])
exp_bg = exp_bg.sort_values("time")

# Loading image data
exp_bg["images"] = [
    pf.loadh5(fname, extra_keys=["alignz", "PAM/FQPDSum"])[0]
    for fname in exp_bg["filename"]
]

dark = np.mean(np.stack(exp_bg["images"]), axis=0)

In [ ]:
# Plot images
fig, ax = cimshow(dark)
fig.set_size_inches(6, 6)
ax.set_title("Dark Image")

### Subtract dark image and normalize with I0

In [ ]:
# Incident intensity
fig, ax = plt.subplots()
ax.plot(np.arange(len(exp)), exp["diode_sum_mean"])
ax.set_xlabel("Image Index")
ax.set_ylabel("Incident Intensity (a.u.)")

In [ ]:
# Execute
images = np.stack(exp.images) - dark
images = images / np.broadcast_to(np.array(exp["diode_sum_mean"]), images.T.shape).T
im_mean = np.mean(images, axis=0)

# Plot images
fig, ax = cimshow(images)
fig.set_size_inches(6, 6)
ax.set_title("Images of scan")

# Preprocessing

## Draw beamstop mask

In [ ]:
poly_mask = interactive.draw_polygon_mask(im_mean)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.coordinates
mask = poly_mask.full_mask.astype(int)

cimshow(mask)

print("Mask coordinates: %s" % p_coord)

In [ ]:
def load_poly_coordinates(polyname):
    if polyname == "bs_cross":
        coord = [
            (946.9258289571189, 901.9078818510652),
            (-13.52166457167948, 915.628560330048),
            (-8.033393180086364, 1107.7180590358075),
            (944.1816932613224, 1099.4856519484179),
            (946.9258289571189, 1988.5856173865054),
            (945.4332858438797, 2042.780748713184),
            (1134.4167167610876, 2056.6765892218023),
            (1145.5333891679823, 1092.305257923697),
            (2007.0755007023129, 1106.2010984323151),
            (2067.2340362524137, 1100.0546685633194),
            (2067.2340362524137, 910.7093055533562),
            (1147.9485781605636, 902.4768984659665),
            (1153.4368495521567, -38.761645192255855),
            (955.8590794548038, -22.296831017476507),
        ]

    return coord

In [ ]:
# Which drawn masks do you want to load?
polygon_names = ["bs_cross"]
mask = load_poly_masks(polygon_names, images[0].shape)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
mi, ma = np.percentile(im_mean, [1, 99])
ax[0].imshow(im_mean * (1 - mask), vmin=mi, vmax=ma)
ax[0].set_title("(1-mask)")
ax[1].imshow(im_mean * mask, vmin=mi, vmax=ma)
ax[1].set_title("mask")

## Find center

### Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. 

In [ ]:
# Set center position via widget
ic = interactive.InteractiveCenter(im_mean)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

### Azimuthal integrator widget for finetuning

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = interactive.AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Plotting to find  relevant q range
I_t, q_t, phi_t = ai.integrate2d(
    im_mean,
    200,
    radial_range=(0, 0.05),
    unit="q_nm^-1",
    correctSolidAngle=False,
    dummy=np.nan,
    mask=mask,
)
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.nanpercentile(I_t, [1, 99])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

# Vertical lines
# q_lines = [0.025, 0.05]
# for qt in q_lines:
#    ax.axvline(qt, ymin=0, ymax=180, c="red")

In [ ]:
aic = interactive.AzimuthalIntegrationCenter(
    # np.log10(images[36] - np.min(images[36]) + 1),
    images[36],
    ai,
    c0=center[0],
    c1=center[1],
    mask=mask,
    im_data_range=[1, 90],
    radial_range=(0.002, 0.03),
    qlines=[40, 60],
)

In [ ]:
# Get center positions
center = [aic.c0, aic.c1]
print(f"Center:", center)

# Azimuthal Integration

In [ ]:
# Setup final azimuthal integrator for virtual geometry
ai = interactive.AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Do 2d Azimuthal integration of all images and add to xarray
list_i2d = []
for im in tqdm(images):
    i2d, q, chi = ai.integrate2d(im, 500, 90, dummy=np.nan, mask=mask)
    list_i2d.append(i2d)

# Setup xarray
data = xr.Dataset()
data["images"] = xr.DataArray(images, dims=["index", "y", "x"])
data[scan_axis] = xr.DataArray(exp[scan_axis], dims=["index"])
data["q"] = q
data["chi"] = chi
data["i2d"] = xr.DataArray(list_i2d, dims=["index", "chi", "q"])
data = data.assign_attrs({"center": center})

## Select relevant chi-range

In [ ]:
# Plot 2d and 1d azimuthal integration to estimate the relevant chi and q range
# which image to show?
idx = 26

# Select chi-range
# sel = (data.chi > -40) * (data.chi < 125) + (data.chi < -76) * (data.chi > 100)
# data["i1d"] = data.i2d.where(sel, drop=True).mean("chi")
data["i1d"] = data.i2d.mean("chi")

# Plot
fig, ax = plt.subplots(
    2,
    1,
    figsize=(8, 8),
    sharex=True,
)
mi, ma = np.nanpercentile(I_t, [0.1, 90])
data["i2d"][idx].plot.imshow(ax=ax[0], vmin=mi, vmax=ma)
ax[0].set_title(f"2d Azimuthal integration")
ax[0].grid()

# Plot 1d azimuthal integration to estimate the relevant q-range
ax[1].plot(data.q, data.i1d[idx])
ax[1].set_yscale("log")
ax[1].set_title("1d Azimuthal Integration")
ax[1].grid()
ax[1].set_ylabel("Integrated intensity")
ax[1].set_xlabel("q")

## Select relevant q-range

In [ ]:
# Select relevant q-range for averaging
q0, q1 = 0.005, 0.04
binning = False
bins = []

# Get SAXS from q-range
sel = (data.q > q0) * (data.q < q1)
data["saxs"] = data.i1d.where(sel, drop=True).mean("q")

# Averaging of same scan axis values or binning
if binning is True:
    # Execute binning
    data_bin = data.groupby_bins(scan_axis, bins).mean()

    # Rename binned values, drop intervals as those cannot be save in h5
    bin_scan_axis = scan_axis + "_bins"
    data_bin = data_bin.swap_dims({bin_scan_axis: scan_axis})
    data_bin = data_bin.drop(bin_scan_axis)
else:
    _, count = np.unique(data[scan_axis].values, return_counts=True)
    if np.any(count > 1):
        data_bin = data.groupby(scan_axis).mean()

# Add scan identifier
data_bin["scan"] = scan

# Add AI mask
data_bin["mask"] = xr.DataArray(mask, dims=["y", "x"])

# Direction of "time"
if np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) >= 0:
    data_bin["order"] = 1
elif np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) < 0:
    data_bin["order"] = -1

# Plot
fig, ax = plt.subplots()
ax.plot(
    data_bin[scan_axis].values,
    data_bin["saxs"].values,
    "o-",
    label="Azimuthal Integration",
)
# ax.plot(
#    data_bin[scan_axis].values,
#    np.mean(data_bin["images"].values * (1 - mask), axis=(1, 2)),
#    label="Simple Mean",
# )

ax.set_xlabel(scan_axis)
ax.set_ylabel("Integrated SAXS")
ax.set_title("Scan: %s" % scan)
ax.grid()
ax.legend()

# Save fig
fname = join(fsave, "Hysteresis_%s_%s.png" % (scan, USER))
plt.savefig(fname)

# Hysteresis Plot

## Select roi for plotting

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(data_bin["images"].values)

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi = np.array([int(y1), int(y2), int(x1), int(x2)])
roi_s = np.s_[roi[0] : roi[1], roi[2] : roi[3]]
print(f"Roi:", roi)

## Plotting

In [ ]:
# Find max and min considering all images
allmin, allmax = np.nanpercentile(data_bin["i2d"].values, [3, 97])
print("Min: %d Max: %d" % (allmin, allmax))

# Create folder for gif single frames
folder_gif = create_folder(join(fsave, "Scan_%s" % scan))

im_fnames = []
for i in tqdm(range(len(data_bin[scan_axis].values))):
    # Plot for averaged image
    fig = plt.figure(figsize=(6.5, 10))
    gs1 = gridspec.GridSpec(
        2,
        1,
        figure=fig,
        left=0.2,
        bottom=0.05,
        right=0.975,
        top=1.1,
        wspace=0,
        hspace=0,
        height_ratios=[3, 1],
    )

    ax0 = fig.add_subplot(gs1[0])
    tmp = data_bin["images"][i].values
    m = ax0.imshow(tmp[roi_s], vmin=allmin, vmax=allmax)
    ax0.set_title(f"%s:  %s = %s" % (scan_id, scan_axis, data_bin[scan_axis].values[i]))
    plt.colorbar(m, ax=ax0, pad=0.045, location="bottom")

    ax1 = fig.add_subplot(gs1[1])
    ax1.plot(data_bin[scan_axis].values, data_bin["saxs"].values)
    ax1.scatter(
        data_bin[scan_axis].values[i], data_bin["saxs"].values[i], 20, color="r"
    )
    ax1.set_xlabel(scan_axis)
    ax1.set_ylabel("Integrated intensity")
    ax1.grid()

    # Save
    fname = join(folder_gif, "Hysteresis_%s_%03d_%s.png" % (scan, i, USER))
    im_fnames.append(fname)
    plt.savefig(fname)
    plt.close()


# Create gif for 1d AI
var = [imageio.imread(file) for file in im_fnames]
fname = f"SAXS_%s_%s.gif" % (scan, USER)
gif_path = join(fsave, fname)
print("Saving gif:%s" % gif_path)
imageio.mimsave(gif_path, var, fps=2)
print("Done!")

In [ ]:
# Drop images
data_bin = data_bin.drop_vars(["images"])

# Save log
folder = join(fsave, "Logs")
create_folder(folder)
fname = join(folder, "Log_Hysteresis_Scan_%03d_%s.nc" % (scan_id, USER))

print(f"Saving:", fname)
data_bin.to_netcdf(fname)

# Batch processing

## Loading and pre-processing

In [ ]:
def worker(
    samplefolder,
    scan,
    mask,
    ai,
    scan_axis,
    binning=None,
    keys=None,
    sort=False,
):
    # Load scan data
    datafolder = join(basefolder, samplefolder)
    exp = preprocess_exp_OF(datafolder, keys=keys, sort=sort)

    # Load background images
    exp_bg = preprocess_exp_BG(datafolder, keys=keys)

    # Normalize images
    dark = np.mean(np.stack(exp_bg["images"]), axis=0)
    images = np.stack(exp.images) - dark
    images = images / np.broadcast_to(np.array(exp["diode_sum_mean"]), images.T.shape).T

    # Create xarray dataset
    data = xr.Dataset()
    data["images"] = xr.DataArray(images, dims=["index", "y", "x"])
    data[scan_axis] = xr.DataArray(exp[scan_axis], dims=["index"])

    # Do 2d Azimuthal integration of all images and append them to list
    list_i2d = []
    for im in tqdm(data["images"].values):
        i2d, q, chi = ai.integrate2d(im, 500, 90, dummy=np.nan, mask=mask)
        list_i2d.append(i2d)

    # Add to xarray
    data["q"] = q
    data["chi"] = chi
    data["i2d"] = xr.DataArray(list_i2d, dims=["index", "chi", "q"])
    data["i1d"] = data.i2d.mean("chi")

    # Averaging of same scan axis values or binning
    if binning is None:
        _, count = np.unique(data[scan_axis].values, return_counts=True)
        if np.any(count > 1):
            data_bin = data.groupby(scan_axis).mean()
    else:
        # Execute binning
        data_bin = data.groupby_bins(scan_axis, bins).mean()

        # Rename binned values, drop intervals as those cannot be save in h5
        bin_scan_axis = scan_axis + "_bins"
        data_bin = data_bin.swap_dims({bin_scan_axis: scan_axis})
        data_bin = data_bin.drop(bin_scan_axis)

    # Add AI mask
    data_bin["mask"] = xr.DataArray(mask, dims=["y", "x"])

    # Add file scan labels
    data_bin["scan"] = scan

    # Direction of "time"
    if np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) >= 0:
        data_bin["order"] = 1
    elif np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) < 0:
        data_bin["order"] = -1

    # Drop images to save disk space
    data_save = data_bin.drop_vars(["images"])

    # Save log
    folder = join(fsave, "Logs")
    fname = join(folder, "Log_SAXS_%s_%s.nc" % (scan, USER))
    print(f"Saving:", fname)
    data_save.to_netcdf(fname)

    return data_bin

In [ ]:
# Name of scans (see pad)
scans = [
    "I0_HystScan_007",
    "I0_HystScan_Inverted_008",
    "I0_HystScan_Inverted_011",
    "I0_HystScan_Inverted_012",
]

# Analysis options
bins = []

# Setup xarray for scans
data_scans = []

# Loop over scans
for scan in tqdm(scans):
    # Process data
    samplefolder = join(sample, scan)
    data = worker(
        samplefolder,
        scan,
        mask,
        ai,
        scan_axis,
        binning=None,
        keys=extra_keys,
        sort=False,
    )

    # Add to xarray list
    data_scans.append(data)

# Combine separate xarrays
data_scans = xr.concat(data_scans, dim="scanid")
data_scans

## Calc and plot SAXS

In [ ]:
# Do you want to norm the hysteresis?
normalization = True

# Select relevant q-range for averaging
# q0, q1 = 0.003, 0.04

# Get SAXS from q-range
sel = (data_scans.q > q0) * (data_scans.q < q1)
data_scans["saxs"] = data_scans.i1d.where(sel, drop=True).mean("q")

# Plot all SAXS images individually
for scanid in data_scans["scanid"].values:
    fig, ax = plt.subplots()

    # Normalize?
    if normalization is True:
        tmp_data = norm(data_scans["saxs"][scanid].values)
    else:
        tmp_data = data_scans["saxs"][scanid].values

    ax.plot(data_scans[scan_axis].values, tmp_data, "o-")
    ax.set_xlabel(scan_axis)
    ax.set_ylabel("Integrated SAXS")
    ax.set_title("Scan: %s" % data_scans["scan"][scanid].values)
    ax.grid()

    # Save fig
    fname = join(fsave, "SAXS_%s_%s.png" % (data_scans["scan"][scanid].values, USER))
    print("Saving: %s" % fname)
    plt.savefig(fname)
    plt.close()

# Plot them together
fig, ax = plt.subplots()
for scanid in data_scans["scanid"].values:
    # Normalize?
    if normalization is True:
        tmp_data = norm(data_scans["saxs"][scanid].values)
    else:
        tmp_data = data_scans["saxs"][scanid].values

    ax.plot(
        data_scans[scan_axis].values,
        tmp_data,
        "o-",
        label=data_scans["scan"][scanid].values,
    )

ax.set_xlabel(scan_axis)
ax.set_ylabel("Integrated SAXS")
ax.grid()
ax.legend()

# Save fig
fname = join(
    fsave,
    "SAXS_%s_%s_%s.png"
    % (data_scans["scan"][0].values, data_scans["scan"][-1].values, USER),
)
plt.savefig(fname)

In [ ]:
# Export gif for all scans individually
for scan_id in data_scans["scanid"].values:
    if data_scans["order"][scan_id] == 1:
        tmp_xr = data_scans.copy()
    elif data_scans["order"][scan_id] == -1:
        tmp_xr = data_scans.reindex(magnet_mean=data_scans.magnet_mean[::-1])

    # Find max and min considering all images
    allmin, allmax = np.nanpercentile(tmp_xr["i2d"][scan_id].values, [3, 97])
    print("Min: %d Max: %d" % (allmin, allmax))

    # Create folder for gif single frames
    folder_gif = create_folder(join(fsave, "Scan_%s" % tmp_xr["scan"][scan_id].values))

    # Normalize?
    if normalization is True:
        tmp_data = norm(tmp_xr["saxs"][scan_id].values)
    else:
        tmp_data = tmp_xr["saxs"][scan_id].values

    im_fnames = []
    for i in tqdm(range(len(tmp_xr[scan_axis].values))):
        # Plot for averaged image
        fig = plt.figure(figsize=(6.5, 10))
        gs1 = gridspec.GridSpec(
            2,
            1,
            figure=fig,
            left=0.2,
            bottom=0.05,
            right=0.975,
            top=1.1,
            wspace=0,
            hspace=0,
            height_ratios=[3, 1],
        )

        ax0 = fig.add_subplot(gs1[0])
        tmp = tmp_xr["images"][scan_id][i].values
        m = ax0.imshow(tmp[roi_s], vmin=allmin, vmax=allmax)
        ax0.set_title(
            f"%s:  %s = %s"
            % (tmp_xr["scan"][scanid].values, scan_axis, tmp_xr[scan_axis].values[i])
        )
        plt.colorbar(m, ax=ax0, pad=0.045, location="bottom")

        ax1 = fig.add_subplot(gs1[1])
        ax1.plot(tmp_xr[scan_axis].values, tmp_data)
        ax1.scatter(
            tmp_xr[scan_axis].values[i],
            tmp_data[i],
            20,
            color="r",
        )
        ax1.set_xlabel(scan_axis)
        ax1.set_ylabel("Integrated intensity")
        ax1.grid()

        # Save
        fname = join(
            folder_gif,
            "SAXS_%s_%03d_%s.png" % (tmp_xr["scan"][scan_id].values, i, USER),
        )
        im_fnames.append(fname)
        plt.savefig(fname)
        plt.close()

    # Create gif for 1d AI
    var = [imageio.imread(file) for file in im_fnames]
    fname = f"SAXS_%s_%s.gif" % (tmp_xr["scan"][scan_id].values, USER)
    gif_path = join(fsave, fname)
    print("Saving gif:%s" % gif_path)
    imageio.mimsave(gif_path, var, fps=2)
    print("Done!")